In [2]:
import pandas as pd
import os, pathlib
import numpy as np
from import_clean import *
from calculate import *
from itertools import combinations

In [3]:
all_data = cleanup_all(import_all_data("utla"), remove_geography_code=False)

c:\Users\keo88794\Documents\GitHub\work_experience_notebooks\import_clean.py:107: UserWarning: File census2021-ts001-utla.csv not found in data/census2021-ts001
  warnings.warn(f"File {folder}-{region}.csv not found in data/{folder}")
c:\Users\keo88794\Documents\GitHub\work_experience_notebooks\import_clean.py:107: UserWarning: File census2021-ts003-utla.csv not found in data/census2021-ts003
  warnings.warn(f"File {folder}-{region}.csv not found in data/{folder}")
c:\Users\keo88794\Documents\GitHub\work_experience_notebooks\import_clean.py:107: UserWarning: File census2021-ts006-utla.csv not found in data/census2021-ts006
  warnings.warn(f"File {folder}-{region}.csv not found in data/{folder}")
c:\Users\keo88794\Documents\GitHub\work_experience_notebooks\import_clean.py:107: UserWarning: File census2021-ts007a-utla.csv not found in data/census2021-ts007a
  warnings.warn(f"File {folder}-{region}.csv not found in data/{folder}")
c:\Users\keo88794\Documents\GitHub\work_experience_noteboo

In [10]:
coefficients = []
keys = list(all_data.keys())
for k1, k2 in combinations(keys, 2):
    dataframes = [all_data[k1], all_data[k2]]
    columns = [[column for column in df.columns if column != "geography code" and "Total" not in column] for df in dataframes]
    totals = [[column for column in df.columns if "Total" in column][0] for df in dataframes]
    df_merged = dataframes[0].merge(dataframes[1], on="geography code")
    for column, total in zip(columns, totals):
        df_merged[column] = df_merged[column].divide(df_merged[total], axis=0)
    for i in columns[0]:
        for j in columns[1]:
            x, y = dataframes[0][i], dataframes[1][j]
            pearsons = round(np.corrcoef(np.array(x), np.array(y))[0][1], 4)
            coefficients.append(pearsons)


KeyError: "['Country of birth: Europe; measures: Value', 'Country of birth: Europe: United Kingdom; measures: Value', 'Country of birth: Africa; measures: Value', 'Country of birth: Middle East and Asia; measures: Value', 'Country of birth: The Americas and the Caribbean; measures: Value'] not in index"